# **Tratamento inicial e imports - Dataset Vinhos**

In [1]:
import numpy as np
import pandas as pd ## Apenas para testar o programa
from collections import Counter  #Usada uma vez para determinar o elemento que mais aparece

from random import seed     #Gera valores aleatórios
from random import randint
from random import sample

import matplotlib as mat
import matplotlib.pyplot as plt#Visualização de dados
import seaborn as sns 
import statistics as stat
import string as st

import colorsys
plt.style.use('seaborn-talk')
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss


from sklearn import linear_model

In [2]:
df = pd.read_csv('winequality.csv')
df = df.dropna()
print(df.columns)

df.describe()
##Não há grandes discrepâncias

df = df.dropna()



Index(['Unnamed: 0', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality', 'wine_is_red'],
      dtype='object')


# **Regressão Linear - Qualidade do vinho**

In [ ]:
x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()


In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25,
                                                    random_state=0)
 
# Cria o modelo
reg = linear_model.LinearRegression()
 
# Treinar o modelo
reg.fit(x_treino, y_treino)
 
# Coeficientes da regressão
print('Coeficientes: ', reg.coef_)
 
#Score
print('Score: {}'.format(reg.score(x_teste, y_teste)))
 

Coeficientes:  [ 9.08047801e-02 -1.37351929e+00 -1.53945202e-01  4.49227609e-02
 -5.28421310e-01  5.79044964e-03 -2.44155462e-03 -6.36165907e+01
  5.35899364e-01  7.55118293e-01  2.56631220e-01]
Score: 0.2792422072312881


Normalização


In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x_norm, y, test_size=0.25,
                                                    random_state=0)
 
# Cria o modelo
reg = linear_model.LinearRegression()
 
# Treinar o modelo
reg.fit(x_treino, y_treino)
 
# Coeficientes da regressão
print('Coeficientes: ', reg.coef_)
 
#Score
print('Score: {}'.format(reg.score(x_teste, y_teste)))

Coeficientes:  [ 0.11771332 -0.22611397 -0.02236927  0.21371723 -0.01851108  0.1027691
 -0.13799057 -0.19075067  0.08615913  0.11235739  0.30606351]
Score: 0.27924220723126725


Percebe-se que a normalização apresentou um efeito baixo ao prever a qualidade do vinho.

# **KNN sem bibliotecas - vinho tinto ou não**

In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['wine_is_red']

y = y.to_numpy()


In [ ]:

def distancia(x, y): #Calcula a distancia
  dist = np.linalg.norm(x-y)
  return dist

def valor_k (k):
  return k

def previsao (x_teste, x_treino, y_treino, k):
  final_output = []
  for i in range(len(x_teste)):       ## Pega os índices dos dados de teste
    d = []
    votes = []
    for j in range(len(x_treino)):    ##Pega os índices dos dados do treino e compara cada um com o dado teste
      dist = distancia(x_treino[j] , x_teste[i])    #Calcula a distancia
      d.append([dist, j])             ##Registra aqui na lista a distancia e o indice do treino
    d.sort()                          #Ordena a lista e pega apenas até a distancia k
    d = d[0:k]
    for d, j in d:
      votes.append(y_treino[j])        #Pega da lista os valores um por um e coloca o valor de y correspondente do treino
    ans = Counter(votes).most_common(1)[0][0]  #Pega apenas o elemento que mais que aparece
    final_output.append(ans)          #Devolve pra nós o elemento em uma lista.
            
  return final_output

def score(x_teste, x_treino, y_teste, y_treino, k):
  prev = previsao(x_teste, x_treino, y_treino, k)        #Pega a previsão
  cont = 0
  for i in range(len(prev)):
    if prev[i] == y_teste[i]:
      cont += 1
  return cont / len(y_teste)   #Calcula a acurácia. O quanto bateu com o esperado




In [ ]:
#Setamos a variável que vai dividir os dados em treino e teste

tamanho = len(x)
teste = int(0.2*tamanho)
treino = tamanho - teste 
print('Tamanho do set: ', tamanho)
print('Quantidade de teste: ', teste)
print('Quantidade de treino: ', treino)


Tamanho do set:  6497
Quantidade de teste:  1299
Quantidade de treino:  5198


In [ ]:
#Separamos os dados de treino e teste

k = valor_k(2)
x_treino = x[0:treino]
x_teste = x[treino:]
y_treino = y[0:treino]
y_teste = y[treino:]

#Chamamos o KNN

predic = previsao(x_teste, x_treino, y_treino, k)

#Vemos a eficácia do KNN em prever

score = score(x_teste, x_treino, y_teste, y_treino, k)

print(score)


0.9499615088529638


**Normalizando antes **

Aqui já fazemos o uso do Skcit Learn. Para o KNN acima não havíamos usado ainda



In [ ]:
#Normalizar os dados

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste

k = valor_k(2)
x_treino = x_norm[0:treino]
x_teste = x_norm[treino:]
y_treino = y[0:treino]
y_teste = y[treino:]

#Chamamos o KNN

predic = previsao(x_teste, x_treino, y_treino, k)

#Calculamos o score sem chamar a função, pois ela deu erro

prev = predic        
cont = 0
for i in range(len(prev)):
  if prev[i] == y_teste[i]:
    cont += 1
    
score = cont / len(y_teste)

print(score)

0.99153194765204


Podemos observar que o primeiro KNN apresentou um alto nível de acurácia, ficando 94, quase 95%. 

Porém, ao normalizar os dados, percebe-se que essa acurácia chega ao valor de 99%, o que mostra que quanto menor o intervalo entre os valores, mais preciso o KNN vai ser

# **Classificação Binária - Regressão Logística - Vinho tinto ou não**

In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['wine_is_red']

y = y.to_numpy()

In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y , test_size=0.25, random_state=0)

In [ ]:
#Procedemos com a regressão logística

model = LogisticRegression()
model.fit(x_treino, y_treino)

LogisticRegression()

In [ ]:
#Fazemos a previsão

predictions = model.predict(x_teste)

In [ ]:
#Testamos a acurácia

cm = confusion_matrix(y_teste, predictions)

TN, FP, FN, TP = confusion_matrix(y_teste, predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Precisão da classificação binária = {:0.3f}'.format(accuracy))

True Positive(TP)  =  383
False Positive(FP) =  11
True Negative(TN)  =  1211
False Negative(FN) =  20
Precisão da classificação binária = 0.981


**Testando o mesmo código com normalização**

In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_norm, y , test_size=0.25, random_state=0)

In [ ]:
#Procedemos com a regressão logística

model = LogisticRegression()
model.fit(x_treino, y_treino)

LogisticRegression()

In [ ]:
#Fazemos a previsão

predictions = model.predict(x_teste)

In [ ]:
#Testamos a acurácia

cm = confusion_matrix(y_teste, predictions)

TN, FP, FN, TP = confusion_matrix(y_teste, predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Precisão da classificação binária = {:0.3f}'.format(accuracy))

True Positive(TP)  =  401
False Positive(FP) =  8
True Negative(TN)  =  1214
False Negative(FN) =  2
Precisão da classificação binária = 0.994


Percebe-se novamente que a acurácia aumenta quando utilizamos os valores normalizados

# **SVM - Suport Vector Machine - qualidade do vinho**

Vamos testar o modelo para prever a qualidade do vinho, tanto para dados crus e normalizados

In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()

In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y , test_size=0.25, random_state=0)

In [ ]:
#Geramos o modelo te treino

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_treino, y_treino)
svm_predictions = svm_model_linear.predict(x_teste)
 
# Acurácia do método
accuracy = svm_model_linear.score(x_teste, y_teste)
 

In [ ]:
print(accuracy)

0.5304615384615384


**Normalizando**


In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()

In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_norm, y , test_size=0.25, random_state=0)

In [ ]:
#Geramos o modelo te treino

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_treino, y_treino)
svm_predictions = svm_model_linear.predict(x_teste)
 
# Acurácia do método
accuracy = svm_model_linear.score(x_teste, y_teste)
 
print(accuracy)

0.5261538461538462


Para este caso, a normalização na verdade torna o modelo mais impreciso

**OverSample**

In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
x_over, y_over = oversample.fit_resample(x, y)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_over, y_over , test_size=0.25, random_state=0)

In [ ]:
#Geramos o modelo te treino

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_treino, y_treino)
svm_predictions = svm_model_linear.predict(x_teste)
 
# Acurácia do método
accuracy = svm_model_linear.score(x_teste, y_teste)
 
print(accuracy)

0.6792452830188679


Under Sampling

In [3]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()

In [4]:
undersample = NearMiss(version=2, n_neighbors=3)
# transform the dataset
x_under, y_under = undersample.fit_resample(x, y)

In [5]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_under, y_under , test_size=0.25, random_state=0)

In [6]:
#Geramos o modelo te treino

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_treino, y_treino)
svm_predictions = svm_model_linear.predict(x_teste)
 
# Acurácia do método
accuracy = svm_model_linear.score(x_teste, y_teste)
 
print(accuracy)

0.3333333333333333


Percebe-se que o oversample ajuda o modelo, enquanto o undersample piora sua qualidade

# **KNN - Qualidade do vinho**

In [ ]:
#Atribuimos as colunas

x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['quality']

y = y.to_numpy()

In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y , test_size=0.25, random_state=0)

In [ ]:
#Chamamos o método

knn = KNeighborsClassifier(n_neighbors = 7).fit(x_treino, y_treino)
 
# Acurácia do método
accuracy = knn.score(x_teste, y_teste)
print(accuracy)


0.47692307692307695


OverSample

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')
# Aplicamos o método de oversample
x_over, y_over = oversample.fit_resample(x, y)

In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_over, y_over , test_size=0.25, random_state=0)

In [ ]:
##Chamamos o método

knn = KNeighborsClassifier(n_neighbors = 7).fit(x_treino, y_treino)
 
# Acurácia do método
accuracy = knn.score(x_teste, y_teste)
print(accuracy)


0.6333619210977701


UnderSample

In [ ]:
undersample = NearMiss(version=2, n_neighbors=3)
# transform the dataset
x_under, y_under = undersample.fit_resample(x, y)

In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_under, y_under , test_size=0.25, random_state=0)

In [ ]:
##Chamamos o método

knn = KNeighborsClassifier(n_neighbors = 7).fit(x_treino, y_treino)
 
# Acurácia do método
accuracy = knn.score(x_teste, y_teste)
print(accuracy)


0.564923076923077


O método de Oversample se fez bem eficiente, pois foi capaz de aumentar a precisão do modelo em mais de 10 %, já o undersample abaixou e muito a precisão

**Normalizando**


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x_norm, y , test_size=0.25, random_state=0)

In [ ]:
#Chamamos o método

knn = KNeighborsClassifier(n_neighbors = 7).fit(x_treino, y_treino)
 
# Acurácia do método
accuracy = knn.score(x_teste, y_teste)
print(accuracy)


0.564923076923077


Percebe-se que para o KNN trabalhar com dados normalizados melhora a precisão

**Ensemble Learning**

In [ ]:
#Usamos os dados normalizados

In [ ]:
kfold = model_selection.KFold(n_splits=10)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
results = model_selection.cross_val_score(model, x_norm, y, cv=kfold)
print(results.mean())

0.6963195448619177


O método de ensamble learning também melhora o modelo, gerando uma precisão maior até que o Oversample

# **Random Forest - Tinto ou não**

In [ ]:
x = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

x = x.to_numpy()

y = df['wine_is_red']

y = y.to_numpy()

In [ ]:
#Geramos as mesmas variáveis normalizadas

standardizer = StandardScaler()

x_norm = standardizer.fit_transform(x)


In [ ]:
#Separamos os dados em treino e teste, sem normalizar
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y , test_size=0.25, random_state=0)

In [ ]:
#Instaciamos o classificador
rfc = RandomForestClassifier()
rfc.fit(x_treino, y_treino)

y_pred=rfc.predict(x_teste)

print('Score do teste: ', accuracy_score(y_teste,y_pred))

print("Matriz de confusão: \n", confusion_matrix(y_teste,y_pred))

Score do teste:  0.9963076923076923
Matriz de confusão: 
 [[1219    3]
 [   3  400]]


In [ ]:
#Determinar as features que mais importam

rfc.feature_importances_



array([0.03887435, 0.13418514, 0.01122884, 0.0366246 , 0.28137116,
       0.05940192, 0.31333833, 0.04881769, 0.02029082, 0.04644463,
       0.00942252])

Agora vamos testar coisas novas, ver como os parametros influenciam a saída e em seguida aplica-los e ver o que acontece.

In [ ]:
param_grid = {
    "max_depth": [4, 5, 6, 10, 20, 30], ##Profundidade máxima das arvores
    "max_features": ["auto", "sqrt"],
    "n_estimators": [20, 50, 100]   ##Número de árvores a serem testadas
}

cf = RandomForestClassifier()
cf_cv = GridSearchCV(estimator=cf, param_grid=param_grid, cv=3)
cf_cv.fit(x_treino, y_treino)

cf_cv.best_params_

{'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 50}

In [ ]:
#Passa os parametros pro algoritmo

rfc1=RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 50, max_depth=30)

In [ ]:
rfc.fit(x_treino, y_treino)

y_pred=rfc.predict(x_teste)

print('Score do teste: ', accuracy_score(y_teste,y_pred))

Score do teste:  0.9963076923076923


Para o caso não houve grande diferença, mas a principio isso pode ser explicado pela score muito elevado, sendo necessário um modelo quase perfeito para aumentá-lo.